In [8]:
from keras.backend import zeros

from carla_parking import SimulationParking


In [9]:
import cv2

simulation = SimulationParking()
simulation.load_world('Town05')

RuntimeError: time-out of 5000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000

In [3]:

initial_location = {
    'x': 20,
    'y': -30,
    'z': 0.5,
    'yaw': 180
}
location_mirror = {
    'x': 0.5,
    'y': 0.0,
    'z': 1.3
}
vehicle = simulation.init_vehicle('model3', initial_location)
simulation.init_spectator()

In [4]:
import time
import numpy as np

corners_points = [
    simulation.get_location_by_coordinates(6, -28.5, 0),
    simulation.get_location_by_coordinates(6, -31.5, 0),
    simulation.get_location_by_coordinates(11.5, -28.5, 0),
    simulation.get_location_by_coordinates(11.5, -31.5, 0)
]


def my_callback(sensor):
    distances = (
        np.array([
            simulation.calculate_distance_actor_to_location(sensor, point)
            for point in corners_points
        ]))
    angles = (
        np.array([
            simulation.calculate_angle_actor_to_location(sensor, point)
            for point in corners_points
        ]))

    created = time.time()
    tags = f"s=camera_front_mirror_"  # Etiqueta inicial

    # Añadir distancias en el formato d1, d2, ...
    for i, distance in enumerate(distances):
        tags += f"d{i + 1}={round(distance, 3)}_"

    # Añadir ángulos en el formato a1, a2, ...
    for i, angle in enumerate(angles):
        tags += f"a{i + 1}={round(angle, 3)}_"

    # Añadir la marca de tiempo
    tags += f"t={created}"

    return tags

In [5]:
image = zeros((1920, 1080, 3))
camera = simulation.init_camera(
    vehicle, location_mirror, 'rgb', image
)

In [6]:
simulation.aplly_control(vehicle, throttle=0.5, steer=0.0, brake=0.0)

In [7]:
simulation.add_camera_listen_capture_images(
    camera,
    './parking_sequence/',
    my_callback
)
# vehicle.set_autopilot(True)
cv2.waitKey(3000)
vehicle.destroy()
camera.destroy()


True